# KSHE 95 songs to Spotify Playlist

## Scrapping KSHE 95 player

In [1]:
from bs4 import BeautifulSoup
import requests
import json
from operator import itemgetter
import base64
import pandas as pd
import datetime

In [19]:
soup = BeautifulSoup(
    requests.get('http://player.listenlive.co/20101/en/songhistory').text, 
    'html.parser')

In [20]:
song_history = soup.find('section', {'id': ['songHistory']})
song_history_json = json.loads(
    song_history.find('script').text.split(';')[0].split('var songs = ')[-1]
)
song_history_json.sort(key=itemgetter('timestamp'))

In [21]:
song_history_json

[{u'artist': u'JOURNEY',
  u'timestamp': 1502523838000,
  u'title': u"LOVIN',TOUCHIN'/CITY OF THE ANGELS"},
 {u'artist': u'BUSH', u'timestamp': 1502524251000, u'title': u'GLYCERINE'},
 {u'artist': u'AEROSMITH', u'timestamp': 1502524506000, u'title': u'RAG DOLL'},
 {u'artist': u'WHO', u'timestamp': 1502524757000, u'title': u'MY GENERATION'},
 {u'artist': u'VAN HALEN',
  u'timestamp': 1502525198000,
  u'title': u'ERUPTION/YOU REALLY GOT ME'},
 {u'artist': u'LOVERBOY',
  u'timestamp': 1502525458000,
  u'title': u'TURN ME LOOSE'},
 {u'artist': u'JUDAS PRIEST',
  u'timestamp': 1502525800000,
  u'title': u"LIVIN' AFTER MIDNIGHT"},
 {u'artist': u'NEIL YOUNG', u'timestamp': 1502526007000, u'title': u'OLD MAN'},
 {u'artist': u'QUEEN',
  u'timestamp': 1502526204000,
  u'title': u'WE WILL ROCK YOU / WE ARE THE CHAMPIONS'},
 {u'artist': u'THREE DOORS DOWN',
  u'timestamp': 1502526700000,
  u'title': u"WHEN I'M GONE"}]

## Uploading songs to Spotify

### Authentication

In [9]:
user_id = 'ericda'

with open(".spotify-token.json") as f:
    credentials = json.loads(f.read())

In [10]:
auth_header = base64.b64encode(
    '{0}:{1}'.format(credentials['client_id'], credentials['client_secret'])
).encode('ascii')

response = requests.post(
    'https://accounts.spotify.com/api/token',
    headers={'Authorization': 'Basic {0}'.format(auth_header.decode('ascii'))},
    data={'grant_type':'client_credentials'}
)

token = response.json()['access_token']

### Searching a song

In [16]:
def search_track(track_name, artist_name, token, limit=1):
    try:
        response = requests.get(
            'https://api.spotify.com/v1/search',
            params={
                'q':'artist:"{0}"%20track:"{1}"'.format(artist_name, track_name),
                'type':'track',
                'limit': limit
            },
            headers={'Authorization': 'Bearer {0}'.format(token)}
        ).json()['tracks']['items'][0]
    except:
        return {}

    return {
        'song_name': response['name'],
        'artist_name': response['artists'][0]['name'],
        'album_name': response['album']['name'],
        'popularity': response['popularity'],
        'duration_ms': response['duration_ms'],
        'explicit': response['explicit'],
        'spotify_uri': response['uri'],
        'album_image': response['album']['images'][0]['url']
    }

In [17]:
df = pd.DataFrame([search_track(s['title'].lower(), s['artist'].lower(), token) for s in song_history_json])

In [18]:
df

,album_image,album_name,artist_name,duration_ms,explicit,popularity,song_name,spotify_uri
0,https://i.scdn.co/image/6c989970431d5d18cc6db6...,Don't Say No (Remastered Edition),Billy Squier,281453.0,False,58.0,Lonely Is The Night - 2010 - Remaster,spotify:track:28eg5bkYvIHT7HPmzZFwLN
1,https://i.scdn.co/image/7683bcf0385767362266d9...,Paranoid (Remastered Edition),Black Sabbath,355303.0,False,66.0,Iron Man - Remastered Version,spotify:track:3IOQZRcEkplCXg6LofKqE9
2,https://i.scdn.co/image/d1cc1c3d84eb45304a906d...,The Essential Heart,Heart,306373.0,False,42.0,Straight On,spotify:track:47w1ZdXC1h2HvhgQLych1N
3,https://i.scdn.co/image/339704434b2733dbeb4464...,Eliminator,ZZ Top,243066.0,False,52.0,Got Me Under Pressure - 2008 Remastered Version,spotify:track:1jzZZsfzxsebFa6KpWXaAr
4,https://i.scdn.co/image/f43bea738e74d000f14298...,The Grand Illusion,Styx,331866.0,False,45.0,Fooling Yourself (The Angry Young Man),spotify:track:7apWQJSE3gJB55Dp8bXTW9
5,https://i.scdn.co/image/e6e513e68c4eefdfc7bc21...,Led Zeppelin III (Remastered),Led Zeppelin,146250.0,False,74.0,Immigrant Song,spotify:track:78lgmZwycJ3nzsdgmPPGNx
6,https://i.scdn.co/image/9544e5967a8c1bf4561935...,Purple,Stone Temple Pilots,293986.0,False,58.0,Big Empty,spotify:track:07jUylEMp4LBOWZagde8tF
7,https://i.scdn.co/image/b05e4cdc434c1f8b88cc94...,Goodbye Yellow Brick Road (40th Anniversary Ce...,Elton John,295466.0,False,50.0,Saturday Night’s Alright (For Fighting) - Rema...,spotify:track:12yHvSYFXI7PGzNecUvIDu
8,https://i.scdn.co/image/750a4e51bebe3270835c51...,Asia,Asia,227626.0,False,62.0,Heat Of The Moment,spotify:track:2ZBalNGwjqdKJd29O76wBV
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
